<a href="https://colab.research.google.com/github/sujanya001/SAKT_Continual-learning/blob/main/5049___5117___1998_%2B_feat_incr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#5049 -> 5117 -> 1998

In [ ]:
import numpy as np
import pandas as pd
import os
import gc 
import csv
from tqdm import tqdm 
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

import seaborn as sns 
import matplotlib.pyplot as plt  

import torch 
import torch.nn as nn 
import torch.nn.utils.rnn as rnn_utils 
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader 
import torchvision.models as models
from torchsummary import summary
from copy import deepcopy

In [ ]:
MAX_SEQ = 30
epochs = 150
MAX_LR = 2e-3

In [ ]:
dtype = {'school_id': 'int32',
         'user_id': 'int32',
         'question_id': 'int32',
         'correct': 'int16'}
from google.colab import drive
drive.mount('/content/gdrive')


train_df = pd.read_csv('/content/gdrive/MyDrive/skill_builder_data_corrected_collapsed_org.csv', usecols=[20, 3, 5, 7], dtype=dtype, encoding = "ISO-8859-1")


Mounted at /content/gdrive


In [ ]:
schools_5049 = train_df[train_df["school_id"] == 5049]
task1_df=pd.DataFrame(schools_5049)
task1_uniq = task1_df.drop_duplicates(subset = ["problem_id"])
schools_1998 = train_df[train_df["school_id"] == 1998]
task2_df=pd.DataFrame(schools_1998)
task2_uniq = task2_df.drop_duplicates(subset = ["problem_id"])
schools_5117 = train_df[(train_df["school_id"] == 5117)]
task3_df=pd.DataFrame(schools_5117)
task3_uniq = task3_df.drop_duplicates(subset = ["problem_id"])
tasks_df = pd.concat([task1_uniq,task3_uniq,task2_uniq])
tasks_uniq = tasks_df.drop_duplicates(subset = ["problem_id"])
tasks_df.to_csv('out.csv')
train_task = pd.read_csv('/content/out.csv')
print(len(tasks_uniq), len(task1_uniq),len(task2_uniq),len(task3_uniq))
print(train_task[(train_task["school_id"] == 1998) & (train_task["school_id"] != 5049)])

11925 7975 3065 2728
       Unnamed: 0  user_id  problem_id  correct  school_id
10703        1536    80171       51413        1       1998
10704        1537    80171       51425        1       1998
10705        1538    80172       51471        1       1998
10706        1539    80172       51476        1       1998
10707        1540    80173       51392        1       1998
...           ...      ...         ...      ...        ...
13763      324003    85656       46879        1       1998
13764      324004    85656       46880        1       1998
13765      327720    86229       54032        0       1998
13766      327721    86229       54033        1       1998
13767      327722    86229       54034        0       1998

[3065 rows x 5 columns]


In [ ]:
pd.set_option('display.max_rows', None)
a = task1_df['problem_id']
b = task2_df['problem_id']
c = task3_df['problem_id']
print("Overlap questions between 5049 and 1998:",len(pd.Series(list(set(a) & set(b)))))
print("Overlap questions between 1998 and 5117:",len(pd.Series(list(set(b) & set(c)))))
print("Overlap questions between 5049 and 5117:",len(


0


In [ ]:
questions = list(train_task["problem_id"].unique())
n_questions = len(questions)
print("Total no of questions:", n_questions)
count = {}
print(questions)
for i in range(1,len(questions)+1):
    count[questions[i-1]]=i
print(count)
train_task['new_prob_id']=0
for i in range(len(train_task['problem_id'])):
    train_task['new_prob_id'][i]=count[train_task['problem_id'][i]]

Total no of questions: 11925
[51449, 51406, 51412, 51428, 51476, 51425, 51434, 51405, 51443, 51460, 51397, 51401, 51455, 51451, 51394, 51408, 51424, 51440, 51439, 51471, 51466, 51421, 51473, 51435, 51400, 51409, 51429, 51465, 51422, 135656, 135648, 135637, 135634, 135660, 51407, 51480, 51414, 51467, 51456, 51461, 51415, 51395, 51446, 135649, 135652, 51459, 51431, 51457, 51447, 51469, 51416, 51433, 51454, 51479, 51398, 51423, 135645, 135661, 135647, 51436, 51430, 51450, 51475, 51417, 51413, 135669, 51418, 51404, 51396, 51478, 51463, 51452, 51410, 51477, 135670, 135666, 135636, 135642, 135651, 135665, 135641, 135633, 135658, 135632, 135654, 135662, 51472, 51399, 51438, 135664, 135631, 135635, 135659, 51464, 51411, 135653, 135640, 51392, 51453, 51448, 51445, 51402, 51420, 135657, 51458, 51442, 51474, 51441, 135638, 135663, 135655, 92339, 92318, 92319, 92316, 92354, 92334, 92328, 92351, 92338, 92350, 92331, 92348, 92333, 147976, 147947, 147962, 92345, 92349, 92352, 92347, 147972, 147961, 1

In [ ]:
train_task.to_csv('qm_index.csv')
print(len(train_task['new_prob_id']))
print(train_task[train_task.duplicated(["problem_id"], keep=False)])
#train_task = pd.read_csv('/content/qm_index.csv')

13768
       Unnamed: 0  user_id  problem_id  correct  school_id  new_prob_id
0            1361    79411       51449        1       5049            1
1            1362    79411       51406        1       5049            2
2            1363    79414       51412        0       5049            3
3            1364    79414       51428        1       5049            4
4            1365    79414       51476        1       5049            5
...           ...      ...         ...      ...        ...          ...
13761      323993    85656       80672        1       1998         6313
13762      323994    85656       80673        1       1998         6314
13765      327720    86229       54032        0       1998         5884
13766      327721    86229       54033        1       1998         5885
13767      327722    86229       54034        0       1998         5886

[3531 rows x 6 columns]


In [ ]:
'''questions = list(train_df["problem_id"].unique())
schools_tot = train_df["school_id"].unique()
#schools = comb_df["school_id"].unique()
#print("Unique Questions:", questions)
n_questions = len(questions)
#n_schools = len(schools)
#print("number questions:", n_questions)
#print("Tot num of schools:", len(schools_tot))
#print("#of schools = ", n_schools)

#print(type(questions))#.shape())


fzfn= train_df[train_df["school_id"] == 5049]
foos= train_df[train_df["school_id"] == 5117]
onne= train_df[train_df["school_id"] == 1998]
comb_2 = pd.concat([foos, onne])
comb = pd.concat([fzfn, foos, onne])
n_questions_5049 = len(fzfn.problem_id.unique())
n_questions_5117 = len(foos.problem_id.unique())
n_questions_1998 = len(onne.problem_id.unique())
n_questions_comb_2 = len(comb_2.problem_id.unique())
n_questions_comb = len(comb.problem_id.unique())

print(n_questions_5049,n_questions_5117,n_questions_1998,n_questions_comb_2, n_questions_comb)

'questions = list(train_df["problem_id"].unique())\nschools_tot = train_df["school_id"].unique()\n#schools = comb_df["school_id"].unique()\n#print("Unique Questions:", questions)\nn_questions = len(questions)\n#n_schools = len(schools)\n#print("number questions:", n_questions)\n#print("Tot num of schools:", len(schools_tot))\n#print("#of schools = ", n_schools)\n\n#print(type(questions))#.shape())\n\n\nfzfn= train_df[train_df["school_id"] == 5049]\nfoos= train_df[train_df["school_id"] == 5117]\nonne= train_df[train_df["school_id"] == 1998]\ncomb_2 = pd.concat([foos, onne])\ncomb = pd.concat([fzfn, foos, onne])\nn_questions_5049 = len(fzfn.problem_id.unique())\nn_questions_5117 = len(foos.problem_id.unique())\nn_questions_1998 = len(onne.problem_id.unique())\nn_questions_comb_2 = len(comb_2.problem_id.unique())\nn_questions_comb = len(comb.problem_id.unique())\n\nprint(n_questions_5049,n_questions_5117,n_questions_1998,n_questions_comb_2, n_questions_comb)'

In [ ]:
'''count = {}
#print(questions)
for i in range(1,len(questions)+1):
    count[questions[i-1]]=i
#print(count)
train_df['new_prob_id']=0
for i in range(len(train_df['problem_id'])):
    train_df['new_prob_id'][i]=count[train_df['problem_id'][i]]'''

"count = {}\n#print(questions)\nfor i in range(1,len(questions)+1):\n    count[questions[i-1]]=i\n#print(count)\ntrain_df['new_prob_id']=0\nfor i in range(len(train_df['problem_id'])):\n    train_df['new_prob_id'][i]=count[train_df['problem_id'][i]]"

In [ ]:
group = train_task[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))

#pd.set_option('display.max_rows', None)
print("Group:",len(group))


#Task 1 : students with school id = 5049
var= train_task[train_task["school_id"] == 5049]
print("No of schools with 5049:",len(var))
group1 = var[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group1:",len(group1))

#Task 2 : students with school id = 5117
var2= train_task[train_task["school_id"] == 5117]
print("No of schools with 5117:",len(var2))
group3 = var2[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group1:",len(group3))

#Task 3 : students with school id = 1998
var1= train_task[train_task["school_id"] == 1998]
print("No of schools with 1998:",len(var1))
group2 = var1[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group2:",len(group2))

#Task 4 : students with school id = 5049 & 5117
comb_df = pd.concat([var, var2])
print("No of schools with 5049 & 5117: ",len(comb_df))
group_comb = comb_df[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group combined of 5049 and 5117 :",group_comb)

#Task 5 : students with school id = 5117 & 1998
comb_df_2 = pd.concat([var2, var1])
print("No of schools with 5117 & 1998: ",len(comb_df_2))
group_comb_2 = comb_df_2[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group combined of 5117 and 1998 tasks :",group_comb_2)

#Task 6 : students with school id = 5049 & 1998
comb_df_13 = pd.concat([var, var1])
print("No of schools with 5117 & 1998: ",len(comb_df_13))
group_comb_13 = comb_df_13[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group combined of 5049 & 1998 :",group_comb_13)

#Task 7 : students with school id = 5049 & 1998 $ 5117
comb_df3 = pd.concat([var, var1, var2])
print("No of schools with 5049 & 1998 & 5117: ",len(comb_df3))
group_12 = comb_df3[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group combined of 3:",len(group_12))

comb_dffff = pd.concat([comb_df, var1])
print("No of schools with 5049 & 1998 & 5117: ",len(comb_dffff))
group_123 = comb_dffff[['user_id', 'new_prob_id', 'correct']].groupby('user_id').apply(lambda r: (
            r['new_prob_id'].values,
            r['correct'].values))
print("Length of group combined of 3:",len(group_123))



Group: 259
No of schools with 5049: 7975
Length of group1: 93
No of schools with 5117: 2728
Length of group1: 81
No of schools with 1998: 3065
Length of group2: 85
No of schools with 5049 & 5117:  10703
Length of group combined of 5049 and 5117 : user_id
74864                                        ([3228], [1])
79409    ([586, 714, 715, 1097, 1236, 1237, 1238, 1397,...
79410    ([541, 542, 543, 544, 587, 588, 589, 716, 717,...
79411    ([1, 2, 238, 333, 334, 335, 336, 337, 468, 469...
79412    ([471, 472, 473, 474, 590, 724, 725, 726, 1109...
                               ...                        
92126                               ([8109, 8110], [1, 1])
92127                               ([1232, 8111], [1, 1])
92128                                        ([8112], [1])
92129                                        ([1221], [1])
92225              ([17, 18, 68, 28, 10], [1, 0, 1, 0, 1])
Length: 174, dtype: object
No of schools with 5117 & 1998:  5793
Length of group combined of 511

In [ ]:
import random
np.random.seed(1)
torch.manual_seed(1)


In [ ]:

x=0
lis=[]
for i in range(x,len(group1)):
  if(x>len(group1)):
    break
  lis.append(group1[x:x+int(len(group1)/5)])
  x=x+int(len(group1)/5)
#train1 = lis[0].append(lis[5][0:1])t#rain1_lis = 
a = lis[0].append(lis[5][0:1])
b = lis[1].append(lis[5][1:2])
c = lis[2].append(lis[5][2:3])
d = lis[3].append(lis[5][3:4])
e = lis[4]
train1 = pd.concat([b,c,d,e])
#print(len(a),len(b),len(c),len(d),len(e))
print(len(train1))
test1 = a
print(len(test1))

74
19


In [ ]:
''' Run this for normal CL
x=0
lis=[]
for i in range(x,len(group2)):
  if(x>len(group2)):
    break
  lis.append(group2[x:x+int(len(group2)/5)])
  x=x+int(len(group2)/5)
#train1 = lis[0].append(lis[5][0:1])t#rain1_lis = 
a = lis[0].append(lis[5][0:1])
b = lis[1].append(lis[5][1:2])
c = lis[2].append(lis[5][2:3])
d = lis[3].append(lis[5][3:4])
e = lis[4]
train2 = pd.concat([b,c,d,e])
#print(len(a),len(b),len(c),len(d),len(e))
print(len(train2))
test2 = a
print(len(test2))'''

##### Run this for feature incremental learning #####
x=0
lis=[]
for i in range(x,len(group_comb)):
  if(x>len(group_comb)):
    break
  lis.append(group_comb[x:x+int(len(group_comb)/5)])
  x=x+int(len(group_comb)/5)
#train1 = lis[0].append(lis[5][0:1])t#rain1_lis = 
a = lis[0].append(lis[5][0:1])
b = lis[1].append(lis[5][1:2])
c = lis[2].append(lis[5][2:3])
d = lis[3].append(lis[5][3:4])
e = lis[4]
train2 = pd.concat([b,c,d,e])
#print(len(a),len(b),len(c),len(d),len(e))
print(len(train2))
test2 = a
print(len(test2))

136
35


In [ ]:
'''x=0
lis=[]
for i in range(x,len(group3)):
  if(x>len(group3)):
    break
  lis.append(group3[x:x+int(len(group3)/5)])
  x=x+int(len(group3)/5)
#train1 = lis[0].append(lis[5][0:1])t#rain1_lis = 
a = lis[0].append(lis[5][0:1])
b = lis[1].append(lis[5][1:2])
c = lis[2].append(lis[5][2:3])
d = lis[3].append(lis[5][3:4])
e = lis[4]
train3 = pd.concat([b,c,d,e])
#print(len(a),len(b),len(c),len(d),len(e))
print(len(train3))
test3 = a
print(len(test3))'''
##### Run this for feature incremental learning #####

x=0
lis=[]
for i in range(x,len(group_123)):
  if(x>len(group_123)):
    break
  lis.append(group_123[x:x+int(len(group_123)/5)])
  x=x+int(len(group_123)/5)
#train1 = lis[0].append(lis[5][0:1])t#rain1_lis = 
a = lis[0].append(lis[5][0:1])
b = lis[1].append(lis[5][1:2])
c = lis[2].append(lis[5][2:3])
d = lis[3].append(lis[5][3:4])
e = lis[4]
train3 = pd.concat([b,c,d,e])
#print(len(a),len(b),len(c),len(d),len(e))
print(len(train3))
test3 = a
print(len(test3))

199
50


In [ ]:
class SAKTDataset(Dataset):
    def __init__(self, group, n_questions, max_seq=MAX_SEQ):
        super(SAKTDataset, self).__init__()
        self.max_seq = max_seq
        self.n_questions = n_questions
        self.samples = group
        
        self.user_ids = [] 
        for schools in group.index:
            q, qa = group[schools]
            #print("Q:", q)
            #print("QA:", qa)
            if len(q) < 2:
                continue
            self.user_ids.append(schools)
            

    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self, index):
        
        users= train_task['user_id'].unique()
        user_id = self.user_ids[index]

        #print("Index:", index)
        #print("User ID:", user_id)

        q_, qa_ = self.samples[user_id]

        #q_, qa_ = self.samples[users]
        #print("q_, qa_", q_ , qa_)
        
        seq_len = len(q_)
        #print("seq_len", seq_len)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)

        if seq_len >= self.max_seq:
            #if random.random()>0.1:
                #start = random.randint(0,(seq_len-self.max_seq))
                start = seq_len - self.max_seq
                end = start + self.max_seq
                q[:] = q_[start:end]
                qa[:] = qa_[start:end]
            #else:
            #   q[:] = q_[-self.max_seq:]
            #   qa[:] = qa_[-self.max_seq:]
            
        else:
            '''if random.random()>0.1:
                start = 0
                end = random.randint(2,seq_len)
                seq_len = end - start
                q[-seq_len:] = q_[0:seq_len]
                qa[-seq_len:] = qa_[0:seq_len]
            '''  
            #else:
                # The previous case: we store all the elements of q
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_
                   
        target_id = q[1:]
        label = qa[1:]
        #print("Target:", target_id)
        #print("Label:", label)        
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()  
        x += (qa[:-1] == 1) * self.n_questions

        #print("X:",x)
        return x, target_id, label

In [ ]:
#Train dataset and dataloader for 5049
train_dataset_1 = SAKTDataset(train1, n_questions)
train_dataloader_1 = DataLoader(train_dataset_1, batch_size=10, shuffle=True, num_workers=8)

#Train dataset and dataloader for 5049 & 5117
train_dataset_2 = SAKTDataset(train2, n_questions)
train_dataloader_2 = DataLoader(train_dataset_2, batch_size=10, shuffle=True, num_workers=8)

#Train dataset and dataloader for 5049&5117&1998
train_dataset_3 = SAKTDataset(train3, n_questions)
train_dataloader_3 = DataLoader(train_dataset_3, batch_size=10, shuffle=True, num_workers=8)

#Train dataset and dataloader for 5049 & 1998 & 5117
#train_dataset_12 = SAKTDataset(train_12, n_questions)
#train_dataloader_12 = DataLoader(train_dataset_12, batch_size=10, shuffle=True, num_workers=8)

'''
#Validation dataset and dataloader for 5049:
val_dataset_1 = SAKTDataset(val1, n_questions)
val_dataloader_1 = DataLoader(val_dataset_1, batch_size=10, shuffle=True, num_workers=8)

#Validation dataset and dataloader for 1998:
val_dataset_2 = SAKTDataset(val2, n_questions)
val_dataloader_2 = DataLoader(val_dataset_2, batch_size=10, shuffle=True, num_workers=8)

#Validation dataset and dataloader for 5049 & 1998:
val_dataset_12 = SAKTDataset(val12, n_questions)
val_dataloader_12 = DataLoader(val_dataset_12, batch_size=10, shuffle=True, num_workers=8)'''

#Test dataset and dataloader for 5049:
test_dataset_1 = SAKTDataset(test1, n_questions )
test_dataloader_1 = DataLoader(test_dataset_1, batch_size=20, shuffle=False, num_workers=8)

#Test dataset and dataloader for 1998:
test_dataset_2 = SAKTDataset(test2, n_questions )
test_dataloader_2 = DataLoader(test_dataset_2, batch_size=20, shuffle=False, num_workers=8)

#Test dataset and dataloader for 5117:
test_dataset_3 = SAKTDataset(test3, n_questions )
test_dataloader_3 = DataLoader(test_dataset_3, batch_size=20, shuffle=False, num_workers=8)
'''
print(f'Train dataset for 5049: {next(iter(train_dataset_1))}')
print(f'Train dataloader for 5049: {next(iter(train_dataloader_1))}')
print(f'Train dataset for 1998: {next(iter(train_dataset_2))}')
print(f'Train dataloader for 1998: {next(iter(train_dataloader_2))}')
print(f'Test dataset for 5049: {next(iter(test_dataset_1))}')
print(f'Test dataloader for 5049: {next(iter(test_dataloader_1))}')
print(f'Test dataset for 1998: {next(iter(test_dataset_2))}')
print(f'Test dataloader for 1998: {next(iter(test_dataloader_2))}')
print(f'Test dataloader for 5117: {len(test_dataloader_2)}')
print("val:",len(val_dataset_1))
print("va2:",len(val_dataset_2))
print(f'Train dataloader for 5049 & 1998: {next(iter(train_dataloader_12))}')'''
print(len(train_dataset_1),len(train_dataset_2),len(train_dataset_3),len(train_dataloader_2),len(train_dataloader_1),len(test_dataloader_1),len(test_dataloader_2))

72 126 185 13 8 1 2


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
# The first class, FFN, defines a feed-forward network (hence, a MLP with dropout).

class FFN(nn.Module):
    def __init__(self, state_size=200):
        # This NN has only 2 layers. They are densely connected.
        # The first layer has ReLU as activation function.
        # The second layer has Dropout.
        
        super(FFN, self).__init__()
        self.state_size = state_size
            # state_size only refers to the number of inputs. Hence we have the same number of inputs as of hidden neurons and output neurons.
        
        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        x = self.relu(x)

        return self.dropout(x)


def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)      # The we convert the matrix into a tensor (so that PyTorch can process it).

In [ ]:
# This defines the whole SAKT model. Last layer is a FFN network.
class SAKTModel(nn.Module):
    def __init__(self, n_questions, max_seq=120, embed_dim=128):
        super(SAKTModel, self).__init__()
        self.n_questions = n_questions
        self.embed_dim = embed_dim
        
        # First argument `num_embeddings` is the size of the dictionary of embeddings.
        # The second argument `embedding_dim` is the size of each embedding vector.
        # This creates the embeddings: observe that all the embedding layers are created with the torch layer nn.Embedding.
        # from dictionary of questions to dimension d.

        #max_value = comb_df["problem_id"].max()
        #max_value = max(ques)
        #print("MAx:", max_value)
        self.embedding = nn.Embedding(2*n_questions+1, embed_dim)
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_questions+1, embed_dim)

        # This defines the Multihead Attention as in the paper
        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 
        self.ffn = FFN(embed_dim)
        
        # The output is the linear of the outputs of FFN
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        # x is a tensor
        # Calling x.device returns device(type='cuda', index=0)
        device = x.device
        # Transform the tensor using the embedding.

        #import pdb; pdb.set_trace()
        x = self.embedding(x)
       
        # This does several things:
        # Creates a 1D tensor that goes from 0 to k, where k is the second dimension of x
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)
        
        # This creates the embedding of the position
        pos_x = self.pos_embedding(pos_id)
        # We add the position and the input tensor x 
        x = x + pos_x
        
        # Finally, we make an embedding of the question_ids.
        e = self.e_embedding(question_ids)
        
        x = x.permute(1, 0, 2) # x: [bs, s_len, embed] => [s_len, bs, embed]
        e = e.permute(1, 0, 2)
        
        # Creates the mask in order to avoid using the future to predict the past
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)

        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) # att_output: [s_len, bs, embed] => [bs, s_len, embed]

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1), att_weight

In [ ]:
# We define a function that defines each train epoch.
def train_epoch(model, dataloader, optim, scheduler, criterion, device="cpu"):
    model.train()

    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    
    tbar = tqdm(dataloader)

    for item in tbar:
    #for item, (input, target) in enumerate(train_iterator):
        # Gets the first item of train_iterator and sends it to the device and converts it to long type.

        x = item[0].to(device).long()
        target_id = item[1].to(device).long()
        label = item[2].to(device).float()
        target_mask = (target_id != 0)

        # set the gradients to 0.
        optim.zero_grad()
        # Get the output and weights of the model.
        #output, atten_weight = model(x, target_id)
        output, atten_weight = model(x, target_id)
        
        #print("output:", output)
        #print("attention_weight:", atten_weight)      
        
        loss = criterion(output, label)
        loss.backward()
        optim.step()
        scheduler.step()
        train_loss.append(loss.item())


        
        output = torch.masked_select(output, target_mask)
        label = torch.masked_select(label, target_mask)
        
        pred = (torch.sigmoid(output) >= 0.5).long()
        #print(f'Pred {pred}')

        
        num_corrects += (pred == label).sum().item()
        #num_corrects += (output == label).float().sum()
        num_total += len(label)

        labels.extend(label.view(-1).data.cpu().numpy())
        outs.extend(output.view(-1).data.cpu().numpy())

        #tbar.set_description('loss - {:.4f}'.format(loss))
    
    #print(f'Pred: {pred} LAbel: {label}')
    
    
    # Compute Accuracy
    acc = num_corrects / num_total
    roc = roc_auc_score(labels, outs)
    precision, recall, _ = precision_recall_curve(labels,outs)
    auc_score = auc(recall, precision)
    loss = np.average(train_loss)

    #Printing the confusion matrix 
    print(metrics.confusion_matrix(label,pred))
    #Printing the precision and recall, among other metrics
    print(metrics.classification_report(label,pred))

    return loss, acc, roc, auc_score

In [ ]:
#Call the SAKT model

#### For 5049 task ####

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SAKTModel(n_questions, embed_dim = 128)  #The dimension of embeddings d is 128.

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.99, weight_decay=0.01)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()


scheduler_1 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_1), epochs=epochs
)

scheduler_2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_2), epochs=epochs
)

scheduler_3 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_3), epochs=epochs
)

#scheduler_12 = torch.optim.lr_scheduler.OneCycleLR(
#    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_12), epochs=epochs
#)

model.to(device)
criterion.to(device)
print(model)
#print(model_5117)

SAKTModel(
  (embedding): Embedding(23851, 128)
  (pos_embedding): Embedding(119, 128)
  (e_embedding): Embedding(11926, 128)
  (multi_att): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (layer_normal): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (ffn): FFN(
    (lr1): Linear(in_features=128, out_features=128, bias=True)
    (relu): ReLU()
    (lr2): Linear(in_features=128, out_features=128, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (pred): Linear(in_features=128, out_features=1, bias=True)
)


In [ ]:
#Function call for Train on 5049
best_auc = 0
max_steps = 50
step = 0
for epoch in range(epochs):
  train_loss, train_acc, roc, prc_auc = train_epoch(model, train_dataloader_1, optimizer, scheduler_1, criterion, device)
  print("epoch - {}/{} train_loss - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f} ".format(epoch+1, epochs, train_loss, train_acc, roc, prc_auc))
  
  if roc > best_auc:
    best_auc = roc
    step = 0
    torch.save(model.state_dict(), "sakt_model.pt")
  else:
    step += 1
    if step >= max_steps:
      break

  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:01<00:00,  6.75it/s]


[[10 14]
 [12 22]]
              precision    recall  f1-score   support

         0.0       0.45      0.42      0.43        24
         1.0       0.61      0.65      0.63        34

    accuracy                           0.55        58
   macro avg       0.53      0.53      0.53        58
weighted avg       0.55      0.55      0.55        58

epoch - 1/150 train_loss - 0.701 acc - 0.507 roc - 0.492 prc_auc - 0.653 


  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:01<00:00,  6.68it/s]


[[ 5  6]
 [ 8 12]]
              precision    recall  f1-score   support

         0.0       0.38      0.45      0.42        11
         1.0       0.67      0.60      0.63        20

    accuracy                           0.55        31
   macro avg       0.53      0.53      0.52        31
weighted avg       0.57      0.55      0.56        31

epoch - 2/150 train_loss - 0.690 acc - 0.515 roc - 0.494 prc_auc - 0.653 


  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:01<00:00,  7.41it/s]


...

[[ 7 17]
 [ 6 28]]
              precision    recall  f1-score   support

         0.0       0.54      0.29      0.38        24
         1.0       0.62      0.82      0.71        34

    accuracy                           0.60        58
   macro avg       0.58      0.56      0.54        58
weighted avg       0.59      0.60      0.57        58

epoch - 148/150 train_loss - 0.549 acc - 0.673 roc - 0.662 prc_auc - 0.767 


  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:02<00:00,  3.92it/s]


[[ 2  7]
 [ 4 43]]
              precision    recall  f1-score   support

         0.0       0.33      0.22      0.27         9
         1.0       0.86      0.91      0.89        47

    accuracy                           0.80        56
   macro avg       0.60      0.57      0.58        56
weighted avg       0.78      0.80      0.79        56

epoch - 149/150 train_loss - 0.535 acc - 0.678 roc - 0.660 prc_auc - 0.767 


  0%|          | 0/8 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8/8 [00:02<00:00,  3.12it/s]

[[ 3 12]
 [ 4 39]]
              precision    recall  f1-score   support

         0.0       0.43      0.20      0.27        15
         1.0       0.76      0.91      0.83        43

    accuracy                           0.72        58
   macro avg       0.60      0.55      0.55        58
weighted avg       0.68      0.72      0.69        58

epoch - 150/150 train_loss - 0.538 acc - 0.675 roc - 0.659 prc_auc - 0.766 


In [ ]:
#Test function 
import psutil
def test_epoch(model, test_iterator, criterion, device="cpu"):
  model.load_state_dict(torch.load("sakt_model.pt"))
  model.eval()
  #print("inside test_epoch !!") 
  test_loss = []
  num_corrects = 0
  num_total = 0
  labels = []
  outs = []
  preds = []

  prev_test_df = None
  tbar = tqdm(test_iterator)
  for item in tbar:
          x = item[0].to(device).long()
          target_id = item[1].to(device).long()
          label = item[2].to(device).float()
          target_mask = (target_id != 0)

          with torch.no_grad():
            output, att_weight = model(x, target_id)
          
          
          output = torch.masked_select(output, target_mask)
          label = torch.masked_select(label, target_mask)

          loss = criterion(output, label)
          test_loss.append(loss.item())

          pred = (torch.sigmoid(output) >= 0.5).long()
          num_corrects += (pred == label).sum().item()
          num_total += len(label)
          print('num_cor:', num_corrects)
          print("tot:",num_total)

          labels.extend(label.squeeze(-1).data.cpu().numpy())
          outs.extend(output.view(-1).data.cpu().numpy())
          preds.extend(pred.squeeze(-1).data.cpu().numpy())

  
  acc = num_corrects / num_total
  roc = roc_auc_score(labels, outs)
  precision, recall, _ = precision_recall_curve(labels,outs)
  auc_score = auc(recall, precision)
  loss = np.average(test_loss)
  
  print(metrics.confusion_matrix(label,pred))
  print(metrics.classification_report(label,pred))

  return loss, acc, roc, auc_score


In [ ]:
#make test epoch =1 
epoch_test = 1

In [ ]:
#Testing on 5049
for epoch in range(epoch_test):
  test_loss_1, test_acc_1, test_roc_1, prc_auc_1 = test_epoch(model, test_dataloader_1, criterion, device)
  print("Testing on 5049!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_1, test_acc_1, test_roc_1, prc_auc_1))


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1/1 [00:00<00:00,  1.65it/s]

num_cor: 277
tot: 490


100%|██████████| 1/1 [00:00<00:00,  1.12it/s]

[[ 31 165]
 [ 48 246]]
              precision    recall  f1-score   support

         0.0       0.39      0.16      0.23       196
         1.0       0.60      0.84      0.70       294

    accuracy                           0.57       490
   macro avg       0.50      0.50      0.46       490
weighted avg       0.52      0.57      0.51       490

Testing on 5049!!!
epoch - 1/1 test - 0.727 acc - 0.565 roc - 0.487 prc_auc - 0.599


In [ ]:
#Load the model weights and test on task3

In [ ]:
#Call the SAKT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SAKTModel(n_questions, embed_dim = 128)  #The dimension of embeddings d is 128.

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.99, weight_decay=0.01)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

scheduler_1 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_1), epochs=epochs
)

scheduler_2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_2), epochs=epochs
)

scheduler_3 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_3), epochs=epochs
)

#scheduler_12 = torch.optim.lr_scheduler.OneCycleLR(
#    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_12), epochs=epochs
#)

model.to(device)
criterion.to(device)
#print(model)

BCEWithLogitsLoss()

In [ ]:
#Function call for Train on 5117
best_auc = 0
max_steps = 50
step = 0
for epoch in range(epochs):
  train_loss, train_acc, roc, prc_auc = train_epoch(model, train_dataloader_3, optimizer, scheduler_3, criterion, device)
  print("epoch - {}/{} train_loss - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f} ".format(epoch+1, epochs, train_loss, train_acc, roc, prc_auc))
  
  if roc > best_auc:
    best_auc = roc
    step = 0
    torch.save(model.state_dict(), "sakt_task2.pt")
  else:
    step += 1
    if step >= max_steps:
      break

  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19/19 [00:01<00:00, 10.64it/s]


[[13 12]
 [36 37]]
              precision    recall  f1-score   support

         0.0       0.27      0.52      0.35        25
         1.0       0.76      0.51      0.61        73

    accuracy                           0.51        98
   macro avg       0.51      0.51      0.48        98
weighted avg       0.63      0.51      0.54        98

epoch - 1/150 train_loss - 0.600 acc - 0.502 roc - 0.510 prc_auc - 0.660 


  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19/19 [00:01<00:00, 10.66it/s]


[[17 19]
 [34 30]]
              precision    recall  f1-score   support

         0.0       0.33      0.47      0.39        36
         1.0       0.61      0.47      0.53        64

    accuracy                           0.47       100
   macro avg       0.47      0.47      0.46       100
weighted avg       0.51      0.47      0.48       100

epoch - 2/150 train_loss - 0.583 acc - 0.500 roc - 0.510 prc_auc - 0.660 


  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19/19 [00:01<00:00, 11.43it/s]


...

[[ 3 46]
 [ 1 84]]
              precision    recall  f1-score   support

         0.0       0.75      0.06      0.11        49
         1.0       0.65      0.99      0.78        85

    accuracy                           0.65       134
   macro avg       0.70      0.52      0.45       134
weighted avg       0.68      0.65      0.54       134

epoch - 148/150 train_loss - 0.435 acc - 0.651 roc - 0.594 prc_auc - 0.725 


  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19/19 [00:01<00:00, 11.27it/s]


[[12 20]
 [ 6 64]]
              precision    recall  f1-score   support

         0.0       0.67      0.38      0.48        32
         1.0       0.76      0.91      0.83        70

    accuracy                           0.75       102
   macro avg       0.71      0.64      0.66       102
weighted avg       0.73      0.75      0.72       102

epoch - 149/150 train_loss - 0.429 acc - 0.655 roc - 0.596 prc_auc - 0.727 


  0%|          | 0/19 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 19/19 [00:01<00:00, 11.17it/s]

[[ 4 35]
 [ 4 56]]
              precision    recall  f1-score   support

         0.0       0.50      0.10      0.17        39
         1.0       0.62      0.93      0.74        60

    accuracy                           0.61        99
   macro avg       0.56      0.52      0.46        99
weighted avg       0.57      0.61      0.52        99

epoch - 150/150 train_loss - 0.430 acc - 0.653 roc - 0.595 prc_auc - 0.728 


In [ ]:
#Test function 
import psutil
def test_epoch(model, test_iterator, criterion, device="cpu"):
  model.load_state_dict(torch.load("sakt_task2.pt"))
  model.eval()
  #print("inside test_epoch !!") 
  test_loss = []
  num_corrects = 0
  num_total = 0
  labels = []
  outs = []
  preds = []

  prev_test_df = None
  tbar = tqdm(test_iterator)
  for item in tbar:
          x = item[0].to(device).long()
          target_id = item[1].to(device).long()
          label = item[2].to(device).float()
          target_mask = (target_id != 0)

          with torch.no_grad():
            output, att_weight = model(x, target_id)
          
          
          output = torch.masked_select(output, target_mask)
          label = torch.masked_select(label, target_mask)

          loss = criterion(output, label)
          test_loss.append(loss.item())

          pred = (torch.sigmoid(output) >= 0.5).long()
          num_corrects += (pred == label).sum().item()
          num_total += len(label)
          print('num_cor:', num_corrects)
          print("tot:",num_total)

          labels.extend(label.squeeze(-1).data.cpu().numpy())
          outs.extend(output.view(-1).data.cpu().numpy())
          preds.extend(pred.squeeze(-1).data.cpu().numpy())

  
  acc = num_corrects / num_total
  roc = roc_auc_score(labels, outs)
  precision, recall, _ = precision_recall_curve(labels,outs)
  auc_score = auc(recall, precision)
  loss = np.average(test_loss)
  
  print(metrics.confusion_matrix(label,pred))
  print(metrics.classification_report(label,pred))

  return loss, acc, roc, auc_score


In [ ]:
#make test epoch =1 
epoch_test = 1

In [ ]:
#Test on 5049
for epoch in range(epoch_test):
  test_loss_1, test_acc_1, test_roc_1, prc_auc_1 = test_epoch(model, test_dataloader_1, criterion, device)
  print("Testing on 5049!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_1, test_acc_1, test_roc_1, prc_auc_1))


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1/1 [00:00<00:00,  1.83it/s]

num_cor: 294
tot: 490


100%|██████████| 1/1 [00:00<00:00,  1.19it/s]

[[ 27 169]
 [ 27 267]]
              precision    recall  f1-score   support

         0.0       0.50      0.14      0.22       196
         1.0       0.61      0.91      0.73       294

    accuracy                           0.60       490
   macro avg       0.56      0.52      0.47       490
weighted avg       0.57      0.60      0.53       490

Testing on 5049!!!
epoch - 1/1 test - 0.709 acc - 0.600 roc - 0.502 prc_auc - 0.596


In [ ]:
#Test on 5117
for epoch in range(epoch_test):
  test_loss_3, test_acc_3, test_roc_3, prc_auc_3 = test_epoch(model, test_dataloader_3, criterion, device)
  print("Testing on 5117!!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_3, test_acc_3, test_roc_3, prc_auc_3))
  



  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 33%|███▎      | 1/3 [00:00<00:01,  1.69it/s]

num_cor: 294
tot: 501
num_cor: 479
tot: 863
num_cor: 539
tot: 974


100%|██████████| 3/3 [00:00<00:00,  3.36it/s]

[[ 5 37]
 [14 55]]
              precision    recall  f1-score   support

         0.0       0.26      0.12      0.16        42
         1.0       0.60      0.80      0.68        69

    accuracy                           0.54       111
   macro avg       0.43      0.46      0.42       111
weighted avg       0.47      0.54      0.49       111

Testing on 5117!!!!
epoch - 1/1 test - 0.735 acc - 0.553 roc - 0.505 prc_auc - 0.567


In [ ]:
#Load the model weights and test on task2

In [ ]:
#Call the SAKT model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SAKTModel(n_questions, embed_dim = 128)  #The dimension of embeddings d is 128.

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.99, weight_decay=0.01)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

scheduler_1 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_1), epochs=epochs
)

scheduler_2 = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_2), epochs=epochs
)

#scheduler_12 = torch.optim.lr_scheduler.OneCycleLR(
#    optimizer, max_lr=MAX_LR, steps_per_epoch=len(train_dataloader_12), epochs=epochs
#)

model.to(device)
criterion.to(device)
#print(model)

BCEWithLogitsLoss()

In [ ]:
#Function call for Train on 1998
best_auc = 0
max_steps = 50
step = 0
for epoch in range(epochs):
  train_loss, train_acc, roc, prc_auc = train_epoch(model, train_dataloader_2, optimizer, scheduler_2, criterion, device)
  print("epoch - {}/{} train_loss - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f} ".format(epoch+1, epochs, train_loss, train_acc, roc, prc_auc))
  
  if roc > best_auc:
    best_auc = roc
    step = 0
    torch.save(model.state_dict(), "sakt_task2.pt")
  else:
    step += 1
    if step >= max_steps:
      break

  0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 13/13 [00:01<00:00,  8.88it/s]


[[33 29]
 [55 44]]
              precision    recall  f1-score   support

         0.0       0.38      0.53      0.44        62
         1.0       0.60      0.44      0.51        99

    accuracy                           0.48       161
   macro avg       0.49      0.49      0.48       161
weighted avg       0.52      0.48      0.48       161

epoch - 1/150 train_loss - 0.650 acc - 0.509 roc - 0.509 prc_auc - 0.703 


  0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 13/13 [00:01<00:00,  8.94it/s]


[[19 23]
 [52 54]]
              precision    recall  f1-score   support

         0.0       0.27      0.45      0.34        42
         1.0       0.70      0.51      0.59       106

    accuracy                           0.49       148
   macro avg       0.48      0.48      0.46       148
weighted avg       0.58      0.49      0.52       148

epoch - 2/150 train_loss - 0.637 acc - 0.506 roc - 0.506 prc_auc - 0.703 


  0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 13/13 [00:01<00:00,  8.83it/s]


...

[[ 4 29]
 [ 5 71]]
              precision    recall  f1-score   support

         0.0       0.44      0.12      0.19        33
         1.0       0.71      0.93      0.81        76

    accuracy                           0.69       109
   macro avg       0.58      0.53      0.50       109
weighted avg       0.63      0.69      0.62       109

epoch - 148/150 train_loss - 0.428 acc - 0.696 roc - 0.621 prc_auc - 0.768 


  0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 13/13 [00:01<00:00,  8.74it/s]


[[ 5 23]
 [ 7 78]]
              precision    recall  f1-score   support

         0.0       0.42      0.18      0.25        28
         1.0       0.77      0.92      0.84        85

    accuracy                           0.73       113
   macro avg       0.59      0.55      0.54       113
weighted avg       0.68      0.73      0.69       113

epoch - 149/150 train_loss - 0.428 acc - 0.701 roc - 0.622 prc_auc - 0.769 


  0%|          | 0/13 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 13/13 [00:01<00:00,  8.56it/s]

[[11 55]
 [ 9 99]]
              precision    recall  f1-score   support

         0.0       0.55      0.17      0.26        66
         1.0       0.64      0.92      0.76       108

    accuracy                           0.63       174
   macro avg       0.60      0.54      0.51       174
weighted avg       0.61      0.63      0.57       174

epoch - 150/150 train_loss - 0.437 acc - 0.699 roc - 0.621 prc_auc - 0.770 


In [ ]:
#Test function 
import psutil
def test_epoch(model, test_iterator, criterion, device="cpu"):
  model.load_state_dict(torch.load("sakt_task2.pt"))
  model.eval()
  #print("inside test_epoch !!") 
  test_loss = []
  num_corrects = 0
  num_total = 0
  labels = []
  outs = []
  preds = []

  prev_test_df = None
  tbar = tqdm(test_iterator)
  for item in tbar:
          x = item[0].to(device).long()
          target_id = item[1].to(device).long()
          label = item[2].to(device).float()
          target_mask = (target_id != 0)

          with torch.no_grad():
            output, att_weight = model(x, target_id)
          
          
          output = torch.masked_select(output, target_mask)
          label = torch.masked_select(label, target_mask)

          loss = criterion(output, label)
          test_loss.append(loss.item())

          pred = (torch.sigmoid(output) >= 0.5).long()
          num_corrects += (pred == label).sum().item()
          num_total += len(label)
          print('num_cor:', num_corrects)
          print("tot:",num_total)

          labels.extend(label.squeeze(-1).data.cpu().numpy())
          outs.extend(output.view(-1).data.cpu().numpy())
          preds.extend(pred.squeeze(-1).data.cpu().numpy())

  
  acc = num_corrects / num_total
  roc = roc_auc_score(labels, outs)
  precision, recall, _ = precision_recall_curve(labels,outs)
  auc_score = auc(recall, precision)
  loss = np.average(test_loss)
  
  print(metrics.confusion_matrix(label,pred))
  print(metrics.classification_report(label,pred))

  return loss, acc, roc, auc_score


In [ ]:
#make test epoch =1 
epoch_test = 1

In [ ]:
#Test on 5049
for epoch in range(epoch_test):
  test_loss_2, test_acc_2, test_roc_2, prc_auc_2 = test_epoch(model, test_dataloader_1, criterion, device)
  print("Testing on 5049!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_2, test_acc_2, test_roc_2, prc_auc_2))


  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 1/1 [00:00<00:00,  1.87it/s]

num_cor: 279
tot: 490


100%|██████████| 1/1 [00:00<00:00,  1.08it/s]

[[ 17 179]
 [ 32 262]]
              precision    recall  f1-score   support

         0.0       0.35      0.09      0.14       196
         1.0       0.59      0.89      0.71       294

    accuracy                           0.57       490
   macro avg       0.47      0.49      0.43       490
weighted avg       0.50      0.57      0.48       490

Testing on 5049!!!
epoch - 1/1 test - 0.721 acc - 0.569 roc - 0.508 prc_auc - 0.609


In [ ]:
#Test on 5117
for epoch in range(epoch_test):
  test_loss_3, test_acc_3, test_roc_3, prc_auc_3 = test_epoch(model, test_dataloader_3, criterion, device)
  print("Testing on 5117!!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_3, test_acc_3, test_roc_3, prc_auc_3))
  



  0%|          | 0/3 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 67%|██████▋   | 2/3 [00:00<00:00,  3.26it/s]

num_cor: 280
tot: 501
num_cor: 465
tot: 863
num_cor: 533
tot: 974


100%|██████████| 3/3 [00:01<00:00,  2.99it/s]

[[ 3 39]
 [ 4 65]]
              precision    recall  f1-score   support

         0.0       0.43      0.07      0.12        42
         1.0       0.62      0.94      0.75        69

    accuracy                           0.61       111
   macro avg       0.53      0.51      0.44       111
weighted avg       0.55      0.61      0.51       111

Testing on 5117!!!!
epoch - 1/1 test - 0.744 acc - 0.547 roc - 0.493 prc_auc - 0.562


In [ ]:
#Test on 1998
for epoch in range(epoch_test):
  test_loss_1, test_acc_1, test_roc_1, prc_auc_1 = test_epoch(model, test_dataloader_2, criterion, device)
  print("Testing on 1998!!!")
  print("epoch - {}/{} test - {:.3f} acc - {:.3f} roc - {:.3f} prc_auc - {:.3f}".format(epoch+1,epoch_test, test_loss_1, test_acc_1, test_roc_1, prc_auc_1))


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
 50%|█████     | 1/2 [00:00<00:00,  1.78it/s]

num_cor: 313
tot: 577
num_cor: 486
tot: 870


100%|██████████| 2/2 [00:00<00:00,  2.12it/s]

[[ 15 107]
 [ 13 158]]
              precision    recall  f1-score   support

         0.0       0.54      0.12      0.20       122
         1.0       0.60      0.92      0.72       171

    accuracy                           0.59       293
   macro avg       0.57      0.52      0.46       293
weighted avg       0.57      0.59      0.51       293

Testing on 1998!!!
epoch - 1/1 test - 0.745 acc - 0.559 roc - 0.502 prc_auc - 0.567
